# Top of Book Snapshots at One-Minute Intervals

This notebook demonstrates how to capture best bid and ask prices at regular intervals from ITCH 5.0 data.

Top of book snapshots are useful for:
- Creating time series of bid-ask spreads
- Analyzing price movements over time
- Calculating mid-price evolution

This example uses the `S081321-v50-AAPL-SPY.itch50.gz` file generated in the [Extracting Symbols](02_extracting_symbols.ipynb) example.

In [1]:
from pathlib import Path
import datetime
import pandas as pd
from meatpy.itch50 import ITCH50MessageReader, ITCH50MarketProcessor
from meatpy.event_handlers.lob_recorder import LOBRecorder
from meatpy.writers.parquet_writer import ParquetWriter

# Define paths and parameters
data_dir = Path("data")

file_path = data_dir / "S081321-v50-AAPL-SPY.itch50.gz"
outfile_path = data_dir / "spy_tob.parquet"


processor = ITCH50MarketProcessor("SPY", datetime.date(2021, 8, 13))

writer = ParquetWriter(outfile_path)

# We only care about the top of book
tob_recorder = LOBRecorder(writer=writer, max_depth=1)
# We only want every minute. Nasdaq timestamps are in nanoseconds since 12am.
seconds_range = [x * 1_000_000_000 for x in range(34_130, 57_730 + 1, 60)]
tob_recorder.record_timestamps = seconds_range

# Attach the recorders to the processor
processor.handlers.append(tob_recorder)

In [2]:
with ITCH50MessageReader(file_path) as reader:
    for message in reader:
        processor.process_message(message)
writer.close()

In [3]:
df = pd.read_parquet(outfile_path)

df

FileNotFoundError: [Errno 2] No such file or directory: 'data/spy_tob.parquet'